# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [6]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
print(features.shape,labels.shape)

torch.Size([1000, 2]) torch.Size([1000, 1])


调用框架中现有的API来读取数据

In [7]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.3694, -0.0522],
         [-0.1538, -0.2539],
         [ 0.8822, -0.0723],
         [-0.1614,  1.1120],
         [-1.7812, -0.6072],
         [-0.9819, -0.8885],
         [-1.2368,  0.0485],
         [ 1.3377, -2.0371],
         [ 0.6224, -0.3377],
         [ 0.2065,  0.4332]]),
 tensor([[ 3.6462],
         [ 4.7509],
         [ 6.1937],
         [ 0.0987],
         [ 2.7013],
         [ 5.2606],
         [ 1.5583],
         [13.8111],
         [ 6.5855],
         [ 3.1301]])]

使用框架的预定义好的层

In [8]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [9]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [10]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [11]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000439
epoch 2, loss 0.000096
epoch 3, loss 0.000095


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [13]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0005, 0.0004])
b的估计误差： tensor([0.0001])
